# I. Importing libraries and dataset 

In [ ]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings("ignore")

In [35]:
reg_pd = pickle.load(open('pd_model.sav', 'rb'))

In [36]:
with open('feature_list', 'rb') as f:
    features_all_pd = pickle.load(f)

with open('ref_categories', 'rb') as f:
    ref_categories_pd = pickle.load(f)

In [37]:
loan_data_inputs_train = pd.read_csv('X_train.csv')
loan_data_inputs_test = pd.read_csv('X_test.csv')

In [38]:
LGD = pd.read_csv('LGD.csv', index_col=0).iloc[:, 0]
EAD = pd.read_csv('EAD.csv', index_col=0).iloc[:, 0]

In [39]:
loan_data_inputs_pd = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis = 0)
loan_data_inputs_pd = loan_data_inputs_pd.set_index('Unnamed: 0')

In [40]:
loan_data_inputs_pd_temp = loan_data_inputs_pd[features_all_pd]
loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(ref_categories_pd, axis = 1)

# II. Estimating PD for whole data set

In [41]:
reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([0.03072263, 0.09824766, 0.0342913 , ..., 0.03121185, 0.04439572,
       0.05244923])

In [42]:
loan_data_inputs_pd['PD'] = reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]

c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


# III. Calculating EL

In [43]:
loan_data_inputs_pd['EL'] = loan_data_inputs_pd['PD'] * LGD * EAD
loan_data_inputs_pd['EL'].describe()

count    466285.000000
mean       1079.187164
std        1088.473565
min           9.492234
25%         356.388075
50%         706.886312
75%        1403.224562
max       12091.043698
Name: EL, dtype: float64

In [51]:
loan_data_inputs_pd.reset_index(drop=True, inplace=True)
loan_data_inputs_pd[['funded_amnt', 'EL']].head()

,funded_amnt,EL
0,24000,495.592112
1,10000,576.171909
2,20425,389.680684
3,17200,1875.438388
4,8400,948.908144


In [52]:
loan_data_inputs_pd['funded_amnt'].describe()

count    466285.000000
mean      14291.801044
std        8274.371300
min         500.000000
25%        8000.000000
50%       12000.000000
75%       20000.000000
max       35000.000000
Name: funded_amnt, dtype: float64

In [53]:
loan_data_inputs_pd['EL'].sum()

503208786.86658317

In [54]:
loan_data_inputs_pd['funded_amnt'].sum()

6664052450

In [55]:
loan_data_inputs_pd['EL'].sum() / loan_data_inputs_pd['funded_amnt'].sum()

0.07551092831908655